In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
train_dir = r"D:\Atharva\STUDY\I2IT\BE\Academic\DL\Practical\SS_Dataset\raw-img-20241105T101606Z-001\raw-img\training"
test_dir = r"D:\Atharva\STUDY\I2IT\BE\Academic\DL\Practical\SS_Dataset\raw-img-20241105T101606Z-001\raw-img\testing"

# Image data generator for rescaling and augmenting the images
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize image pixel values to between 0 and 1
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resizing images to match ResNet50 input size
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # We don't need to shuffle test data
)
# Load the ResNet50 model, without the top layers (include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of ResNet50 (do not train them)
base_model.trainable = False

# Create a new model on top of ResNet50
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Pooling layer to reduce dimensions
    layers.Dense(1024, activation='relu'),  # Fully connected layer
    layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Low learning rate to start
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=1,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")

# Plotting the training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

# Load and preprocess an image
img_path = r"D:\Atharva\STUDY\I2IT\BE\Academic\DL\Practical\SS_Dataset\raw-img-20241105T101606Z-001\raw-img\training\elefante\e831b30e29f2063ed1584d05fb1d4e9fe777ead218ac104497f5c978a4efb4bb_640.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Rescale

# Predict the class
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Get the class label
class_labels = list(train_generator.class_indices.keys())
predicted_label = class_labels[predicted_class[0]]

translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel", "dog": "cane", "cavallo": "horse", "elephant" : "elefante", "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto", "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo"}
# Display the image and the predicted label
plt.imshow(img)
plt.title(f"Predicted class: {translate[predicted_label]}")
plt.axis('off')  # Hide the axes
plt.show()
